# RANKING V3

## Leer Datos

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Cargar datos
data = pd.read_csv("df_peleas.csv")  
df = pd.DataFrame(data)

In [51]:
df.columns

Index(['Unnamed: 0', 'index', 'Peleador_A', 'Peleador_B', 'DATE', 'CATEGORY',
       'WINNER', 'METHOD', 'TIME', 'ROUND', 'KD_A', 'KD_B', 'SIG_STR_A',
       'SIG_STR_B', 'TD_PORC_A', 'TD_PORC_B', 'SUB_ATT_A', 'SUB_ATT_B',
       'REV_A', 'REV_B', 'CTRL_A', 'CTRL_B', 'TOTAL_STR_A_x', 'TOTAL_STR_A_y',
       'TOTAL_STR_B_x', 'TOTAL_STR_B_y', 'TD_A_x', 'TD_A_y', 'TD_B_x',
       'TD_B_y', 'STR_HEAD_A_x', 'STR_HEAD_A_y', 'STR_HEAD_B_x',
       'STR_HEAD_B_y', 'STR_BODY_A_x', 'STR_BODY_A_y', 'STR_BODY_B_x',
       'STR_BODY_B_y', 'STR_LEG_A_x', 'STR_LEG_A_y', 'STR_LEG_B_x',
       'STR_LEG_B_y', 'STR_DISTANCE_A_x', 'STR_DISTANCE_A_y',
       'STR_DISTANCE_B_x', 'STR_DISTANCE_B_y', 'STR_CLINCH_A_x',
       'STR_CLINCH_A_y', 'STR_CLINCH_B_x', 'STR_CLINCH_B_y', 'STR_GROUND_A_x',
       'STR_GROUND_A_y', 'STR_GROUND_B_x', 'STR_GROUND_B_y', 'TITLE_FIGHT',
       'WOMEN'],
      dtype='object')

## Convertir Fechas y Ordenar

In [53]:
# Convertir la columna DATE a formato datetime y ordenar de más antigua a más reciente
df["DATE"] = pd.to_datetime(df["DATE"], errors='coerce')  
df = df.sort_values(by="DATE", ascending=True)

## Calcular penalizaciones por antigüedad de la pelea

In [55]:
# Función para calcular la importancia del tiempo con caída gradual
def calcular_factor_tiempo(fecha_pelea):
    LAMBDA = 0.2  # Penalización más leve para peleas antiguas
    años_transcurridos = (datetime.today() - fecha_pelea).days / 365
    return np.exp(-LAMBDA * (años_transcurridos ** 1.1))

## Calcular el factor tiempo de pelea

In [57]:
# Función para calcular el tiempo de pelea
def calcular_tiempo(pelea):
    tiempo = pelea["TIME"]
    round_p = (pelea["ROUND"] - 1) * 5 * 60
    return round_p + tiempo

## Calcular factor método

In [59]:
# Función para obtener factor de método
def obtener_factor_metodo(metodo):
    factores = {
        'KO/TKO': 2.5,
        'Submission': 2.4,
        'Decision - Unanimous': 1.5,
        'Decision - Majority': 1,
        'Decision - Split': 0.85,
        "TKO - Doctor's Stoppage": 1.0
    }
    return factores.get(metodo, 1)

## Calcular factor Oponente

In [61]:
# Función para calcular el factor de oponente en un rango de 1 a 5
def calcular_factor_oponente(puntos_ganador, puntos_perdedor, ganador=True):
    if puntos_ganador == 0:
        return 1  # Evita división por cero
    
    ratio = (puntos_perdedor + 100) / (puntos_ganador + 100)
    ratio = max(0.5, min(ratio, 2.0))

    min_ratio = 0.5
    max_ratio = 2.0

    factor_oponente = 1 + 9 * ((np.sqrt(ratio) - np.sqrt(min_ratio)) / (np.sqrt(max_ratio) - np.sqrt(min_ratio)))

    return max(1, min(5, factor_oponente))

## Calcular Ranking

In [75]:
import pandas as pd

# Cargar los datos
df_peleas = pd.read_csv("df_peleas.csv")
df_peleas["DATE"] = pd.to_datetime(df_peleas["DATE"], errors='coerce')

df_peleas = df_peleas.sort_values(by=["DATE"])

# Diccionario de historial
peleadores_historial = {}
df_historico = []

# Columnas a procesar
stats_columns = ["KD", "SIG_STR", "TD_PORC", "SUB_ATT", "REV", "CTRL", "TOTAL_STR_T", "TOTAL_STR_M"]
stats_columns_2= ["TD_T", "TD_M",
"STR_HEAD_T", "STR_HEAD_M",
"STR_BODY_T", "STR_BODY_M",
"STR_LEG_T", "STR_LEG_M",
"STR_DISTANCE_T", "STR_DISTANCE_M",
"STR_CLINCH_T", "STR_CLINCH_M",
"STR_GROUND_T", "STR_GROUND_M"]

def actualizar_estadisticas(peleador, stats_nuevas):
    """
    Actualiza las estadísticas de un peleador manteniendo la media acumulativa.
    """
    if peleador not in peleadores_historial:
        peleadores_historial[peleador] = {stat+"_A": stats_nuevas[stat+"_A"] for stat in stats_columns}
        peleadores_historial[peleador].update({stat+"_R": stats_nuevas[stat+"_R"] for stat in stats_columns})
        peleadores_historial[peleador]["Peleas"] = 1
        peleadores_historial[peleador]["Puntos"] = 0
        peleadores_historial[peleador]["Racha"] = 0
    else:
        peleas_previas = peleadores_historial[peleador]["Peleas"]
        for stat in stats_columns:
            peleadores_historial[peleador][stat+"_A"] = (
                (peleadores_historial[peleador][stat+"_A"] * peleas_previas) + stats_nuevas[stat+"_A"]
            ) / (peleas_previas + 1)
            
            peleadores_historial[peleador][stat+"_R"] = (
                (peleadores_historial[peleador][stat+"_R"] * peleas_previas) + stats_nuevas[stat+"_R"]
            ) / (peleas_previas + 1)
        
        peleadores_historial[peleador]["Peleas"] += 1

# Iterar sobre cada pelea
for _, row in df_peleas.iterrows():
    peleador_a, peleador_b = row["Peleador_A"], row["Peleador_B"]
    fecha_pelea = row["DATE"]
    ganador = row["WINNER"]
    
    stats_a = {stat+"_A": row[stat+"_A"] for stat in stats_columns}
    stats_a.update({stat+"_R": row[stat+"_B"] for stat in stats_columns})
    
    stats_b = {stat+"_A": row[stat+"_B"] for stat in stats_columns}
    stats_b.update({stat+"_R": row[stat+"_A"] for stat in stats_columns})
    
    actualizar_estadisticas(peleador_a, stats_a)
    actualizar_estadisticas(peleador_b, stats_b)
    
    factorTiempo = calcular_factor_tiempo(fecha_pelea)
    factorM = obtener_factor_metodo(row["METHOD"])
    factor_oAW = calcular_factor_oponente(peleadores_historial[peleador_a]["Puntos"], peleadores_historial[peleador_b]["Puntos"], ganador=True)
    factor_oAL = calcular_factor_oponente(peleadores_historial[peleador_a]["Puntos"], peleadores_historial[peleador_b]["Puntos"], ganador=False)
    factor_oBW = calcular_factor_oponente(peleadores_historial[peleador_b]["Puntos"], peleadores_historial[peleador_a]["Puntos"], ganador=True)
    factor_oBL = calcular_factor_oponente(peleadores_historial[peleador_b]["Puntos"], peleadores_historial[peleador_a]["Puntos"], ganador=False)
    
    factorTitleWin = 3 if row["TITLE_FIGHT"] else 1
    factorTitleLoss = 0.3 if row["TITLE_FIGHT"] else 1
    
    racha_A = 1 + 0.15 * peleadores_historial[peleador_a]["Racha"]
    racha_B = 1 + 0.15 * peleadores_historial[peleador_b]["Racha"]
    
    if ganador == 0:
        peleadores_historial[peleador_a]["Racha"] += 1
        peleadores_historial[peleador_b]["Racha"] = 0
        peleadores_historial[peleador_a]["Puntos"] += (10 * factorTitleWin * factorM * factor_oAW * factorTiempo * racha_A)
        peleadores_historial[peleador_b]["Puntos"] -= (10 * factorTitleLoss * factorM * factor_oBL * factorTiempo)
    elif ganador == 1:
        peleadores_historial[peleador_b]["Racha"] += 1
        peleadores_historial[peleador_a]["Racha"] = 0
        peleadores_historial[peleador_b]["Puntos"] += (10 * factorTitleWin * factorM * factor_oBW * factorTiempo * racha_B)
        peleadores_historial[peleador_a]["Puntos"] -= (10 * factorTitleLoss * factorM * factor_oAL * factorTiempo)
    
    peleadores_historial[peleador_a]["Puntos"] = max(0, peleadores_historial[peleador_a]["Puntos"])
    peleadores_historial[peleador_b]["Puntos"] = max(0, peleadores_historial[peleador_b]["Puntos"])
    peleadores_historial[peleador_a]["Ultima_Pelea"] = fecha_pelea
    peleadores_historial[peleador_b]["Ultima_Pelea"] = fecha_pelea
# Convertir el diccionario a un DataFrame



{'KD_A': 0.75,
 'SIG_STR_A': 0.48,
 'TD_PORC_A': 0.35,
 'SUB_ATT_A': 0.75,
 'REV_A': 0.0,
 'CTRL_A': 133.375,
 'KD_R': 0.125,
 'SIG_STR_R': 0.3825,
 'TD_PORC_R': 0.013749999999999998,
 'SUB_ATT_R': 0.125,
 'REV_R': 0.375,
 'CTRL_R': 35.0,
 'Peleas': 8,
 'Puntos': 1186.5179343335567,
 'Racha': 8,
 'Ultima_Pelea': Timestamp('2024-11-02 00:00:00')}

## Eliminar peleadores que no han peleado en más de 3 años

In [ ]:
# Filtrar peleadores inactivos
#df_historico["Fecha"] = pd.to_datetime(df_historico["Fecha"])
#fecha_limite = datetime.today() - timedelta(days=3*365)
#df_historico = df_historico[df_historico["Fecha"] >= fecha_limite]
# Convertir el diccionario a un DataFrame

## Mostrar Top 15 LibraxLibra

In [77]:
df_peleadores = pd.DataFrame.from_dict(peleadores_historial, orient="index")

# Ordenar por puntos y mostrar los 15 mejores
df_top15 = df_peleadores.sort_values(by="Puntos", ascending=False).head(15)

print(df_top15)

                           KD_A  SIG_STR_A  TD_PORC_A  SUB_ATT_A     REV_A  \
Islam Makhachev        0.235294   0.568824   0.511765   0.764706  0.176471   
Jon Jones              0.250000   0.592083   0.548750   0.458333  0.000000   
Dricus Du Plessis      0.444444   0.506667   0.497778   0.666667  0.111111   
Ilia Topuria           0.750000   0.480000   0.350000   0.750000  0.000000   
Merab Dvalishvili      0.071429   0.429286   0.427143   0.285714  0.000000   
Alex Pereira           0.700000   0.651000   0.100000   0.200000  0.000000   
Belal Muhammad         0.000000   0.425263   0.291579   0.157895  0.000000   
Valentina Shevchenko   0.058824   0.575294   0.722353   0.411765  0.058824   
Alexander Volkanovski  0.375000   0.571250   0.259375   0.187500  0.187500   
Tom Aspinall           0.555556   0.698889   0.333333   0.222222  0.000000   
Sean O'Malley          0.461538   0.604615   0.038462   0.230769  0.076923   
Zhang Weili            0.250000   0.585000   0.332500   0.500000

## Mostrar Historial Peleador

In [43]:
# Mostrar el resultado
peleadores_historial["Wang Cong"]

{'Puntos': 14.979079981230205,
 'Peleas': 3,
 'Racha': 1,
 'Ultima_Pelea': Timestamp('2025-02-15 00:00:00')}